In [ ]:
renv::init()
renv::install("rvest")

In [ ]:
html <- rvest::read_html("https://quotes.toscrape.com/")

In [ ]:
quotes <- html |> 
  rvest::html_nodes(".text") |>
  rvest::html_text()

quotes

In [ ]:
file_conn <- file("./data/citacoes.txt", open = "a")

In [ ]:
url_pattern <- "https://quotes.toscrape.com/page/%d"
urls <- sprintf(url_pattern, 1:10)

header <- c("citacao", "autor")
writeLines(paste(header, collapse = ";"), file_conn, sep = "\n")

for(url in urls) {
  html <- rvest::read_html(url)

  quotes <- html |>
    rvest::html_nodes(".text") |>
    rvest::html_text()


  authors <- html |>
    rvest::html_nodes(".author") |>
    rvest::html_text()

  writeLines(paste0(quotes, ";", authors, collapse = "\n"), file_conn, sep = "")
  Sys.sleep(1)
}

In [ ]:
close(file_conn)

In [ ]:
html <- rvest::read_html("https://quotes.toscrape.com/search.aspx")

hidden_inputs <- html |>
  rvest::html_nodes("input[type='hidden']") |>
  (\(x) setNames(
    rvest::html_attr(x, "value"),
    rvest::html_attr(x, "name")
  ))()

response <- httr::POST(
  url = "https://quotes.toscrape.com/filter.aspx",
  httr::add_headers(
    `Content-Type` = "application/x-www-form-urlencoded"
  ),
  body = list(
    author = "Haruki Murakami",
    tag = "thought",
    `__VIEWSTATE` = hidden_inputs["__VIEWSTATE"]
  ),
  encode = "form"
)

In [ ]:
quotes <- response |>
  rvest::read_html() |>
  rvest::html_nodes(".quote .content") |>
  rvest::html_text()

quotes

In [ ]:
html <- rvest::read_html("https://quotes.toscrape.com/search.aspx")

authors <- html |>
  rvest::html_nodes("#author option") |>
  rvest::html_attr("value")

authors <- Filter(\(x) !is.na(x), authors)

hidden_inputs <- html |>
  rvest::html_nodes("input[type='hidden']") |>
  (\(x) setNames(
    rvest::html_attr(x, "value"),
    rvest::html_attr(x, "name")
  ))()

for(author in authors) {
  response <- httr::POST(
    url = "https://quotes.toscrape.com/filter.aspx",
    httr::add_headers(
      `Content-Type` = "application/x-www-form-urlencoded"
    ),
    body = list(
      author = author,
      `__VIEWSTATE` = hidden_inputs["__VIEWSTATE"]
    ),
    encode = "form"
  )

  tags <- response |>
    rvest::read_html() |>
    rvest::html_nodes("#tag option")  |>
    rvest::html_attr("value")

  for(tag in tags) {
    response <- httr::POST(
      url = "https://quotes.toscrape.com/filter.aspx",
      httr::add_headers(
        `Content-Type` = "application/x-www-form-urlencoded"
      ),
      body = list(
        author = author,
        tag = tag,
        `__VIEWSTATE` = hidden_inputs["__VIEWSTATE"]
      ),
      encode = "form"
    )

    quotes <- response |>
      rvest::read_html() |>
      rvest::html_nodes(".quote .content") |>
      rvest::html_text()

    quotes <- Filter(\(x) !identical(x, character(0)), quotes)
    cat(sprintf("Author: %s; Quotes %s; Tag: %s;\n", author, quotes, tag))
  }
}

In [ ]:
session <- rvest::session("https://quotes.toscrape.com/login")

hidden_inputs <- session |>
  rvest::html_nodes("input[type='hidden']") |>
  (\(x) setNames(
    rvest::html_attr(x, "value"),
    rvest::html_attr(x, "name")
  ))()

hidden_inputs


In [ ]:
form <- session |>
  rvest::html_form() |>
  (\(x) x[[1]])() |>
  rvest::html_form_set(
    username = "Username",
    password = "Password"
  )

session_2 <- rvest::session_submit(session, form)
session_2 |> rvest::session_jump_to("https://quotes.toscrape.com") |> rvest::read_html() |> rvest::html_text() |> cat()

In [ ]:
login <- session |>
  rvest::html_form() |>
  (\(x) x[[1]])() |>
  rvest::html_form_set(
    username = "Username",
    password = "Password"
  )

logged_in <- session |> rvest::session_submit(login)

In [1]:
imdb_url <- "https://www.imdb.com/"
top_250_url <- "chart/top/"

top_250_html <- rvest::read_html(paste0(imdb_url, top_250_url))

In [2]:
movie_urls <- top_250_html |>
  rvest::html_nodes(".cli-children a") |>
  rvest::html_attr("href")

movie_urls

[1] "/title/tt0111161/?ref_=chttp_t_1"   
  [2] "/title/tt0068646/?ref_=chttp_t_2"   
  [3] "/title/tt0468569/?ref_=chttp_t_3"   
  [4] "/title/tt0071562/?ref_=chttp_t_4"   
  [5] "/title/tt0050083/?ref_=chttp_t_5"   
  [6] "/title/tt0108052/?ref_=chttp_t_6"   
  [7] "/title/tt0167260/?ref_=chttp_t_7"   
  [8] "/title/tt0110912/?ref_=chttp_t_8"   
  [9] "/title/tt0120737/?ref_=chttp_t_9"   
 [10] "/title/tt0060196/?ref_=chttp_t_10"  
 [11] "/title/tt0109830/?ref_=chttp_t_11"  
 [12] "/title/tt0137523/?ref_=chttp_t_12"  
 [13] "/title/tt0167261/?ref_=chttp_t_13"  
 [14] "/title/tt1375666/?ref_=chttp_t_14"  
 [15] "/title/tt0080684/?ref_=chttp_t_15"  
 [16] "/title/tt0133093/?ref_=chttp_t_16"  
 [17] "/title/tt0099685/?ref_=chttp_t_17"  
 [18] "/title/tt0073486/?ref_=chttp_t_18"  
 [19] "/title/tt0114369/?ref_=chttp_t_19"  
 [20] "/title/tt0038650/?ref_=chttp_t_20"  
 [21] "/title/tt0047478/?ref_=chttp_t_21"  
 [22] "/title/tt9362722/?ref_=chttp_t_22"  
 [23] "/title/tt0816692/?ref_=chttp_t_23"  
 [24] "/title/tt0102926/?ref_=chttp_t_24"  
 [25] "/title/tt0120815/?ref_=chttp_t_25"  
 [26] "/title/tt0317248/?ref_=chttp_t_26"  
 [27] "/title/tt0118799/?ref_=chttp_t_27"  
 [28] "/title/tt0120689/?ref_=chttp_t_28"  
 [29] "/title/tt0076759/?ref_=chttp_t_29"  
 [30] "/title/tt0103064/?ref_=chttp_t_30"  
 [31] "/title/tt0088763/?ref_=chttp_t_31"  
 [32] "/title/tt0245429/?ref_=chttp_t_32"  
 [33] "/title/tt0253474/?ref_=chttp_t_33"  
 [34] "/title/tt0054215/?ref_=chttp_t_34"  
 [35] "/title/tt6751668/?ref_=chttp_t_35"  
 [36] "/title/tt0172495/?ref_=chttp_t_36"  
 [37] "/title/tt0110357/?ref_=chttp_t_37"  
 [38] "/title/tt0110413/?ref_=chttp_t_38"  
 [39] "/title/tt0120586/?ref_=chttp_t_39"  
 [40] "/title/tt0407887/?ref_=chttp_t_40"  
 [41] "/title/tt2582802/?ref_=chttp_t_41"  
 [42] "/title/tt0482571/?ref_=chttp_t_42"  
 [43] "/title/tt0114814/?ref_=chttp_t_43"  
 [44] "/title/tt0095327/?ref_=chttp_t_44"  
 [45] "/title/tt0056058/?ref_=chttp_t_45"  
 [46] "/title/tt0034583/?ref_=chttp_t_46"  
 [47] "/title/tt15398776/?ref_=chttp_t_47" 
 [48] "/title/tt1675434/?ref_=chttp_t_48"  
 [49] "/title/tt0027977/?ref_=chttp_t_49"  
 [50] "/title/tt0095765/?ref_=chttp_t_50"  
 [51] "/title/tt0064116/?ref_=chttp_t_51"  
 [52] "/title/tt0047396/?ref_=chttp_t_52"  
 [53] "/title/tt0078748/?ref_=chttp_t_53"  
 [54] "/title/tt0021749/?ref_=chttp_t_54"  
 [55] "/title/tt0078788/?ref_=chttp_t_55"  
 [56] "/title/tt0209144/?ref_=chttp_t_56"  
 [57] "/title/tt1853728/?ref_=chttp_t_57"  
 [58] "/title/tt0082971/?ref_=chttp_t_58"  
 [59] "/title/tt0910970/?ref_=chttp_t_59"  
 [60] "/title/tt0405094/?ref_=chttp_t_60"  
 [61] "/title/tt0043014/?ref_=chttp_t_61"  
 [62] "/title/tt0050825/?ref_=chttp_t_62"  
 [63] "/title/tt4154756/?ref_=chttp_t_63"  
 [64] "/title/tt0081505/?ref_=chttp_t_64"  
 [65] "/title/tt0032553/?ref_=chttp_t_65"  
 [66] "/title/tt0051201/?ref_=chttp_t_66"  
 [67] "/title/tt4633694/?ref_=chttp_t_67"  
 [68] "/title/tt0090605/?ref_=chttp_t_68"  
 [69] "/title/tt0361748/?ref_=chttp_t_69"  
 [70] "/title/tt1345836/?ref_=chttp_t_70"  
 [71] "/title/tt0169547/?ref_=chttp_t_71"  
 [72] "/title/tt0057012/?ref_=chttp_t_72"  
 [73] "/title/tt0364569/?ref_=chttp_t_73"  
 [74] "/title/tt2380307/?ref_=chttp_t_74"  
 [75] "/title/tt0086879/?ref_=chttp_t_75"  
 [76] "/title/tt0114709/?ref_=chttp_t_76"  
 [77] "/title/tt0082096/?ref_=chttp_t_77"  
 [78] "/title/tt0112573/?ref_=chttp_t_78"  
 [79] "/title/tt4154796/?ref_=chttp_t_79"  
 [80] "/title/tt7286456/?ref_=chttp_t_80"  
 [81] "/title/tt0119698/?ref_=chttp_t_81"  
 [82] "/title/tt0119217/?ref_=chttp_t_82"  
 [83] "/title/tt5311514/?ref_=chttp_t_83"  
 [84] "/title/tt0087843/?ref_=chttp_t_84"  
 [85] "/title/tt1187043/?ref_=chttp_t_85"  
 [86] "/title/tt0057565/?ref_=chttp_t_86"  
 [87] "/title/tt0045152/?ref_=chttp_t_87"  
 [88] "/title/tt8267604/?ref_=chttp_t_88"  
 [89] "/title/tt0180093/?ref_=chttp_t_89"  
 [90] "/title/tt0435761/?ref_=chttp_t_90"  
 [91] "/title/tt0091251/?ref_=chttp_t_91" 

In [3]:
file_conn <- file("./data/citacoes.txt", open = "a")

In [4]:
header <- c("title", "rating", "score", "director")
writeLines(paste(header, collapse = ";"), file_conn, sep = "\n")

for (movie_url in movie_urls) {
  movie_html <- rvest::read_html(paste0(imdb_url, movie_url))

  title <- movie_html |>
    rvest::html_nodes("h1") |>
    rvest::html_text()

  rating <- movie_html |>
    rvest::html_nodes("h1") |>
    rvest::html_text()

  score <- movie_html |>
    rvest::html_nodes("h1") |>
    rvest::html_text()

  director <- movie_html |>
    rvest::html_nodes("h1") |>
    rvest::html_text()


  line <- c(title, rating, score, director)
  writeLines(paste(line, collapse = ";"), file_conn, sep = "\n")
  # Sys.sleep(1)
}

In [ ]:
close(file_conn)